In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.0-preview2/spark-3.0.0-preview2-bin-hadoop3.2.tgz
!tar xvzf spark-3.0.0-preview2-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyarrow==0.16.*
try:
  # %tensorflow_version only exists in Colab.
  !pip install -q tf-nightly
except Exception:
  pass


spark-3.0.0-preview2-bin-hadoop3.2/
spark-3.0.0-preview2-bin-hadoop3.2/data/
spark-3.0.0-preview2-bin-hadoop3.2/data/streaming/
spark-3.0.0-preview2-bin-hadoop3.2/data/streaming/AFINN-111.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/sample_binary_classification_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/sample_kmeans_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/sample_multiclass_classification_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/sample_lda_libsvm_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/iris_libsvm.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/pagerank_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/sample_linear_regression_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/pic_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/als/
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/als/test.data
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/als/sample_movielens_rati

     |████████████████████████████████| 517.5MB 31kB/s 
     |████████████████████████████████| 3.0MB 45.6MB/s 
     |████████████████████████████████| 460kB 61.4MB/s 


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-preview2-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [ ]:
import tensorflow as tf

In [ ]:
import torchvision 
import torch.nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [ ]:
import io

from keras.applications.imagenet_utils import decode_predictions
import pandas as pd
from pyspark.sql.functions import col, pandas_udf, PandasUDFType
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image



In [ ]:
from sklearn.metrics import f1_score
 

In [ ]:
from pyspark.sql.functions import col, pandas_udf, regexp_extract

In [ ]:
import pathlib
import tensorflow_datasets.public_api as tfds
data_dir = tf.keras.datasets.mnist.load_data("/tmp/mnist")

In [ ]:
(x_train,y_train),(x_test,y_test) = data_dir

In [ ]:
from pyspark.sql.types import *
from PIL import Image
from pyspark.ml.image import ImageSchema
import numpy
x_test = x_test.reshape((x_test.shape[0], -1))
#x_test = Image.fromarray(numpy.uint8(x_test))
print(x_test.shape)
data = [(x_test[i].astype(float).tolist(), str(y_test[i])) for i in range(len(y_test))]
schema = StructType([StructField("image",  ArrayType(FloatType())),
                     StructField("label", StringType())])
df = spark.createDataFrame(data, ["image","label"])

x_train = x_train.reshape((x_train.shape[0], -1))
Train = [(x_train[i].astype(float).tolist(), str(y_train[i])) for i in range(len(y_train))]
schema = StructType([StructField("image",  ArrayType(FloatType())),
                     StructField("label", StringType())])
train = spark.createDataFrame(Train, ["image","label"])

(10000, 784)


In [ ]:
class ImageNetDataset(Dataset):
  """
  Converts image contents into a PyTorch Dataset with standard ImageNet preprocessing.
  """
  def __init__(self, contents):
    self.contents = contents

  def __len__(self):
    return len(self.contents)

  def __getitem__(self, index):
    return self._preprocess(self.contents[index])

  def _preprocess(self, content):
    """
    Preprocesses the input image content using standard ImageNet normalization.
    
    See https://pytorch.org/docs/stable/torchvision/models.html.
    """
    #image = Image.open(io.BytesIO(content))
    image = content.reshape((28,28))
    im  = numpy.zeros((numpy.shape(image)[0],numpy.shape(image)[1],3))
    im[:,:,0] = image
    im[:,:,1] = image
    im[:,:,2] = image
    image = im
    image = Image.fromarray(numpy.uint8(image))
    print(image.size)
    transform = transforms.Compose([
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return transform(image)

In [ ]:
feature_extract = False
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
model_ft = models.squeezenet1_1(pretrained=True)
set_parameter_requires_grad(model_ft, feature_extract)
model_ft.classifier[1] = torch.nn.Conv2d(512, 10, kernel_size=(1,1), stride=(1,1))
model_ft.num_classes = 10
input_size = 224
dataset = torchvision.datasets.MNIST(
    root='PATH',
    train=True,
    transform=transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.expand(3, -1, -1))]),download = True)

loader = DataLoader(
    dataset,
    batch_size=100,
    shuffle=True,
    num_workers=2
)
device = torch.device("cuda")
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = torch.nn.CrossEntropyLoss()(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100== 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
model = model_ft.to(device)
optimizer = optim.Adadelta(model.parameters(), lr=1)

scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(1,21):
    train(model, device,loader, optimizer, epoch)
    scheduler.step()
# data, target = next(iter(loader))
# output = model(data)
# print(output.shape)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.487244
Train Epoch: 1 [10000/60000 (17%)]	Loss: 2.299123
Train Epoch: 1 [20000/60000 (33%)]	Loss: 2.306813
Train Epoch: 1 [30000/60000 (50%)]	Loss: 1.609519
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.596523
Train Epoch: 1 [50000/60000 (83%)]	Loss: 0.400617
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.295587
Train Epoch: 2 [10000/60000 (17%)]	Loss: 0.203495
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.265925
Train Epoch: 2 [30000/60000 (50%)]	Loss: 0.086017
Train Epoch: 2 [40000/60000 (67%)]	Loss: 0.238945
Train Epoch: 2 [50000/60000 (83%)]	Loss: 0.150184
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.103773
Train Epoch: 3 [10000/60000 (17%)]	Loss: 0.041503
Train Epoch: 3 [20000/60000 (33%)]	Loss: 0.012649
Train Epoch: 3 [30000/60000 (50%)]	Loss: 0.017916
Train Epoch: 3 [40000/60000 (67%)]	Loss: 0.018550
Train Epoch: 3 [50000/60000 (83%)]	Loss: 0.018792
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.023504
Train Epoch: 4 [10000/60000 (17%)]	Loss: 0.050791
Train Epoch: 4 [2000

In [ ]:
def imagenet_model_udf(model_fn):
  """
  Wraps an ImageNet model into a Pandas UDF that makes predictions.
  
  You might consider the following customizations for your own use case:
    - Tune DataLoader's batch_size and num_workers for better performance.
    - Use GPU for acceleration.
    - Change prediction types.
  """
  def predict(content_series_iter):
    model = model_fn()
    model.eval()
    for content_series in content_series_iter:
      dataset = ImageNetDataset(list(content_series))
      loader = DataLoader(dataset, batch_size=64)
      with torch.no_grad():
        for image_batch in loader:
          image_batch = image_batch.to(device)
          predictions = model(image_batch)
          predicted_labels = predictions.argmax(dim=1, keepdim=True)
          print(predicted_labels.size())#[x[0] for x in decode_predictions(predictions, top=1)]
          yield pd.DataFrame(predicted_labels)
  return_type = "class: bigInt" #
  return pandas_udf(return_type, PandasUDFType.SCALAR_ITER)(predict)

In [ ]:
mobilenet_v2_udf = imagenet_model_udf(lambda: model)

In [ ]:
#images = table("ml_tmp.flowers")
#predictions = images.withColumn("prediction", mobilenet_v2_udf(col("content")))


predictions =df.withColumn("prediction", mobilenet_v2_udf(col("Image")))
#predictions = mobilenet_v2_udf(data)
display(predictions.select(col("label"), col("prediction")).limit(5))

DataFrame[label: string, prediction: struct<class:bigint>]

In [ ]:
predictions.select(col("label"),col("prediction")).show(5)

+-----+----------+
|label|prediction|
+-----+----------+
|    7|       [7]|
|    2|       [2]|
|    1|       [1]|
|    0|       [0]|
|    4|       [4]|
+-----+----------+
only showing top 5 rows



In [ ]:
predn = predictions.select("*").toPandas()

In [ ]:
result = 0
for i in range(predn.shape[0]):
  if int(predn['prediction'][i][0]) == int(predn['label'][i]):
    result += 1

In [ ]:
print(result/predn.shape[0]) 